In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [ ]:
def min_variance_portfolio(cov_matrix):
    num_assets = cov_matrix.shape[0]
    w0 = np.ones(num_assets) / num_assets

    def portfolio_variance(weights):
        return weights.T @ cov_matrix @ weights
    
    constraints = ({
        'type': 'eq',
        'fun': lambda w: np.sum(w) - 1
    })
    
    bounds = tuple((0, 1) for _ in range(num_assets))
    
    result = minimize(portfolio_variance, w0, method='SLSQP', bounds=bounds, constraints=constraints)
    
    if result.success:
        return result.x
    else:
        raise ValueError("Optimization did not converge")

weights_pp = min_variance_portfolio(cov_pp)
weights_tr = min_variance_portfolio(cov_tr)

print("Minimum Variance Portfolio Weights at datePP:")
print(weights_pp)

print("\nMinimum Variance Portfolio Weights at dateTr:")
print(weights_tr)


Minimum Variance Portfolio Weights at datePP:
[4.26369449e-02 6.29419754e-02 0.00000000e+00 0.00000000e+00
 3.47068351e-02 0.00000000e+00 1.55902498e-01 2.25263735e-01
 2.05260741e-01 3.69924140e-02 1.69177736e-01 0.00000000e+00
 6.71171213e-02 1.64615771e-16]

Minimum Variance Portfolio Weights at dateTr:
[4.26369451e-02 6.29419755e-02 0.00000000e+00 0.00000000e+00
 3.47068354e-02 0.00000000e+00 1.55902499e-01 2.25263734e-01
 2.05260740e-01 3.69924138e-02 1.69177735e-01 5.67579837e-17
 6.71171221e-02 6.32225392e-17]


In [ ]:
def erc_portfolio(cov_matrix):
    num_assets = cov_matrix.shape[0]
    w0 = np.ones(num_assets) / num_assets

    def risk_contribution(weights):
        portfolio_variance = weights.T @ cov_matrix @ weights
        marginal_contrib = cov_matrix @ weights
        risk_contrib = weights * marginal_contrib
        return risk_contrib, portfolio_variance

    def objective(weights):
        rc, _ = risk_contribution(weights)
        return np.sum((rc[:, None] - rc[None, :])**2)

    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = tuple((0, 1) for _ in range(num_assets))

    result = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints)

    if result.success:
        return result.x
    else:
        raise ValueError("ERC optimization did not converge.")

weights_erc_pp = erc_portfolio(cov_pp)
weights_erc_tr = erc_portfolio(cov_tr)

print("ERC Portfolio Weights at datePP:")
print(weights_erc_pp)

print("\nERC Portfolio Weights at dateTr:")
print(weights_erc_tr)

ERC Portfolio Weights at datePP:
[0.04392967 0.05881712 0.03601366 0.04700351 0.05010939 0.03985069
 0.16497572 0.13675447 0.13069228 0.04073866 0.09123861 0.02533302
 0.1047699  0.02977331]

ERC Portfolio Weights at dateTr:
[0.04392967 0.05881712 0.03601366 0.04700351 0.05010939 0.03985069
 0.16497572 0.13675447 0.13069228 0.04073866 0.09123861 0.02533302
 0.1047699  0.02977331]


In [ ]:
def diversification_entropy(weights, cov_matrix):
    eigvals, eigvecs = np.linalg.eigh(cov_matrix)
    Gw = np.dot(np.sqrt(np.diag(eigvals)), np.dot(eigvecs.T, weights))
    var_contrib = Gw**2
    p = var_contrib / np.sum(var_contrib)
    p = np.where(p == 0, 1e-10, p)
    entropy = -np.sum(p * np.log(p))
    Nent = np.exp(entropy)
    return -Nent 

def optimize_min_enb(cov_matrix):
    num_assets = cov_matrix.shape[0]
    w0 = np.ones(num_assets) / num_assets
    
    constraints = ({
        'type': 'eq',
        'fun': lambda w: np.sum(w) - 1
    })
    
    bounds = tuple((0, 1) for _ in range(num_assets))
    
    result = minimize(diversification_entropy, w0, args=(cov_matrix,),
                      method='SLSQP', bounds=bounds, constraints=constraints)
    
    if result.success:
        return result.x
    else:
        raise ValueError("Optimization did not converge.")

weights_enb_pp = optimize_min_enb(cov_pp)
weights_enb_tr = optimize_min_enb(cov_tr)

print("Minimum ENB Portfolio Weights at datePP:")
print(weights_enb_pp)

print("\nMinimum ENB Portfolio Weights at dateTr:")
print(weights_enb_tr)

Minimum ENB Portfolio Weights at datePP:
[7.10267622e-02 6.91594299e-02 4.97539183e-13 1.56081938e-12
 3.41076514e-02 1.31205875e-12 2.27520304e-01 2.38719468e-01
 1.78243014e-01 0.00000000e+00 1.62903744e-01 1.60431376e-12
 1.83196269e-02 2.40052072e-12]

Minimum ENB Portfolio Weights at dateTr:
[7.10267622e-02 6.91594299e-02 4.97539183e-13 1.56081938e-12
 3.41076514e-02 1.31205875e-12 2.27520304e-01 2.38719468e-01
 1.78243014e-01 0.00000000e+00 1.62903744e-01 1.60431376e-12
 1.83196269e-02 2.40052072e-12]
